<a href="https://colab.research.google.com/github/PawanKalyanJada/NLI/blob/main/sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/SNLI_TrainSet.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/SNLI_ValidationSet.csv")

In [ ]:
train_df.drop(['Unnamed: 0'],axis=1,inplace=True)
valid_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
train_df=train_df[train_df.similarity != "-"]
valid_df=valid_df[valid_df.similarity != "-"]

In [ ]:
train_df=train_df.reset_index(drop=True)
valid_df=valid_df.reset_index(drop=True)

In [ ]:
len(train_df),len(valid_df)

(49947, 9842)

In [ ]:
k=[]
for i in range(len(train_df)):
  if(train_df['similarity'][i]=='neutral'):
    k.append(0)
  elif(train_df['similarity'][i]=='contradiction'):
    k.append(1)
  else:
    k.append(2)
k1=[]
for i in range(len(valid_df)):
  if(valid_df['similarity'][i]=='neutral'):
    k1.append(0)
  elif(valid_df['similarity'][i]=='contradiction'):
    k1.append(1)
  else:
    k1.append(2)

In [ ]:
import numpy as np
y=np.array(k)
y1=np.array(k1)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 35.7MB/s 
     |████████████████████████████████| 901kB 37.7MB/s 


In [ ]:
from transformers import TFDistilBertModel
from transformers import DistilBertTokenizer

In [ ]:
t=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
m=TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from keras.models import Model
from keras.layers import Input,Dense,Bidirectional,Dropout,GlobalAveragePooling1D
a=Input(shape=(88,),dtype='int32')
b=Input(shape=(88,),dtype='int32')
emb=m(a,attention_mask=b)[0]
p=Bidirectional(CuDNNLSTM(50,return_sequences=True))(emb)
p=GlobalAveragePooling1D()(p)
p=Dense(100,activation='relu')(p)
p=Dropout(0.1)(p)
out=Dense(64,activation='relu')(p)
m1=Model(inputs=[a,b],outputs=out)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
m.trainable=False

In [ ]:
m1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
tf_distil_bert_model (TFDistilB TFBaseModelOutput(la 66362880    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 88, 100)      328000      tf_distil_bert_model[0][0]   

In [ ]:
from keras.layers import Subtract
k1=Input(shape=(88,),dtype='int32')
k2=Input(shape=(88,),dtype='int32')
k3=Input(shape=(88,),dtype='int32')
k4=Input(shape=(88,),dtype='int32')
out1=Subtract()([m1([k1,k2]),m1([k3,k4])])
out1=Dense(100,activation='relu')(out1)
out1=Dropout(0.2)(out1)
out1=Dense(3,activation='softmax')(out1)
model=Model(inputs=[k1,k2,k3,k4],outputs=out1)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 88)]         0                                            
____________________________________________________________________________________________

In [ ]:
p=train_df['sentence1'].to_list()
p1=train_df['sentence2'].to_list()

In [ ]:
d=t(p,max_length=88,padding=True,truncation=True,pad_to_max_length=True)
d1=t(p1,max_length=88,padding=True,truncation=True,pad_to_max_length=True)

In [ ]:
q1=np.array(d['input_ids'])
q2=np.array(d['attention_mask'])
q4=np.array(d1['input_ids'])
q5=np.array(d1['attention_mask'])

In [ ]:
p2=valid_df['sentence1'].to_list()
p3=valid_df['sentence2'].to_list()

In [ ]:
d2=t(p2,max_length=88,padding=True,truncation=True,pad_to_max_length=True)
d3=t(p3,max_length=88,padding=True,truncation=True,pad_to_max_length=True)

In [ ]:
q7=np.array(d2['input_ids'])
q8=np.array(d2['attention_mask'])
q10=np.array(d3['input_ids'])
q11=np.array(d3['attention_mask'])

In [ ]:
q7.shape

(9842, 65)

In [ ]:
num=np.zeros((9842,23))

In [ ]:
q7=np.concatenate((q7,num),axis=1)
q8=np.concatenate((q8,num),axis=1)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 88)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 88)]         0                                            
____________________________________________________________________________________________

In [ ]:
import keras
model.compile(optimizer=keras.optimizers.Adam(6e-5,amsgrad=True),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit([q1,q2,q4,q5],y,epochs=1,batch_size=32,validation_data=([q7,q8,q10,q11],y1))

1561/1561 [==============================] - 295s 178ms/step - loss: 0.8821 - accuracy: 0.5907 - val_loss: 0.8395 - val_accuracy: 0.6245


In [ ]:
res=model.predict([q7,q8,q10,q11])

In [ ]:
res

array([[0.23808348, 0.33601263, 0.42590392],
       [0.26638636, 0.2324059 , 0.5012077 ],
       [0.25610405, 0.2700822 , 0.47381374],
       ...,
       [0.15424505, 0.58589476, 0.25986016],
       [0.27132046, 0.28143403, 0.44724554],
       [0.46481958, 0.28345773, 0.25172266]], dtype=float32)

In [ ]:
r=np.argmax(res,axis=1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y1,r,digits=3))

              precision    recall  f1-score   support

           0      0.688     0.669     0.678      3219
           1      0.699     0.578     0.633      3237
           2      0.656     0.783     0.713      3368

    accuracy                          0.678      9824
   macro avg      0.681     0.677     0.675      9824
weighted avg      0.681     0.678     0.675      9824



In [ ]:
sen='you should calm down'
sen1='the world is round'
di=t(sen)
di1=t(sen1)

In [ ]:
v=np.array(di['input_ids'])
v1=np.array(di['attention_mask'])
v2=np.array(di1['input_ids'])
v3=np.array(di1['attention_mask'])

In [ ]:
v2

array([   0,  627,  232,   16, 1062,    2])

In [ ]:
num=np.zeros((88-len(v)))
num1=np.zeros((88-len(v2)))
v=np.concatenate((v,num))
v1=np.concatenate((v1,num))
v2=np.concatenate((v2,num1))
v3=np.concatenate((v3,num1))

In [ ]:
v2.shape

(88,)

In [ ]:
res=model.predict([[v.reshape(1,-1),v1.reshape(1,-1),v2.reshape(1,-1),v3.reshape(1,-1)]])

In [ ]:
res

array([[0.36197028, 0.38965595, 0.24837376]], dtype=float32)